In [5]:
### Project Description

In [ ]:
## image maniupulation
import numpy as np
import cv2

import torch

##pretrained model
from torchvision import models

In [1]:
def detect_ai_artifacts(image):
    ##Check for unnatural edges/smoothness (common in AI images).
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)      #convert coloured image to black and white
    edges = cv2.Canny(gray, 100, 200)                   #detect sharp intensity changes
    edge_score = np.mean(edges)                          # find average edge strength
    return edge_score < 30                               # a Lower edge_score = smoother (likely AI)

In [ ]:

class ImageProcessor:
    def __init__(self, image_path):
        self.image_path = image_path         #initialize with path to the image
        self.image = self._load_image()     # Load image immediately on initialization
    
    #loading the image
    def _load_image(self):
        try:
            image = cv2.imread(self.image_path)   #laod the image using opencv
            if image is None:
                raise FileNotFoundError
            return image
        except Exception as e:
            print(f"Error: {e}")
            return None
    
     #preprocess the image for the model
    def preprocess(self, target_size=(224, 224)):
        
        resized = cv2.resize(self.image, target_size)     #resize image(into CNN format)      
        normalized = resized / 255.0                      # Scale pixel values(to values b/n 0-1)
        return np.expand_dims(normalized, axis=0)        # Add batch dimension (models expect input in batches) batch_shape: ( (1, height, width, channels))


processor = ImageProcessor("C:/Users/tette/OneDrive/Documents/AI_Image_Detection/image2.jpg")
processed_img = processor.preprocess()
print(processed_img.shape)  

(1, 224, 224, 3)


In [ ]:
class AIDetector:
    def __init__(self):
        self.model = models.resnet18(pretrained=True)
        self.model.eval()
    
    def predict(self, input_tensor):
        """Use feature-based approach instead of classification."""
        with torch.no_grad():               
            tensor = torch.from_numpy(input_tensor).float()
            tensor = tensor.permute(0, 3, 1, 2)
            
            # Get features from the model (before final classification)
            features = self.model.avgpool(self.model.layer4(
                self.model.layer3(self.model.layer2(self.model.layer1(
                    self.model.maxpool(self.model.relu(self.model.bn1(self.model.conv1(tensor))))
                )))
            )).flatten()
            
            # Use feature variance as AI indicator
            # AI images often have more uniform features
            feature_variance = torch.var(features).item()
            
            return "AI" if feature_variance < 0.5 else "Real"

In [ ]:
class AIDetector:
    #Detect AI images using deep features
    
    def __init__(self):
        #Initialize with ResNet18 (pretrained on ImageNet).
        # Load model with pretrained weights
        self.model = models.resnet18(pretrained=True)
        
        # Set to evaluation mode (disables dropout/batchnorm updates)
        self.model.eval()  
    
    def predict(self, input_tensor):
        ##Predict using feature variance analysis.
        
        with torch.no_grad():       # dont do gradient calculation(no training)
            # Convert arrays to PyTorch tensor and reshape 
            tensor = torch.from_numpy(input_tensor).float()
            tensor = tensor.permute(0, 3, 1, 2)  # Channels-first
            
            # Extract features from intermediate layers
            x = self.model.conv1(tensor)
            x = self.model.bn1(x)
            x = self.model.relu(x)
            x = self.model.maxpool(x)
            
            x = self.model.layer1(x)
            x = self.model.layer2(x)
            x = self.model.layer3(x)
            x = self.model.layer4(x)
            
            # Get final features before classification
            features = self.model.avgpool(x).flatten()
            
            #Calculate feature variance
            feature_variance = torch.var(features).item()
            
            #Variance less than 0.5 implies its AI generated 
            return "AI" if feature_variance < 0.5 else "Real"